In [2]:
from langchain_neo4j import Neo4jGraph

In [5]:
graph = Neo4jGraph(
                url="bolt://localhost:7687",
                username="neo4j",
                password="neo4jpassword",
                refresh_schema=False
            )

graph

In [ ]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0, model_name="gpt-4-turbo")

llm_transformer = LLMGraphTransformer(llm=llm)

In [10]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
from gemma import gm  # Assuming you have GemmaModel imported from the Gemma library

llm = gm.nn.Gemma3_4B()

# llm_transformer = LLMGraphTransformer(llm=llm)

ModuleNotFoundError: No module named 'grain'